<a href="https://colab.research.google.com/github/AntoinettedeLima/IRP_FYP_20210522/blob/main/Wind_Data_collection_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
// Define area of interest (update coordinates as needed)
var geometry = ee.Geometry.Rectangle([-113.5, 44.5, -108.5, 49.5]);

// Load ERA5 dataset for wind data
var era5 = ee.ImageCollection("ECMWF/ERA5/DAILY")
              .filterBounds(geometry)
              .filterDate('2015-04-11', '2024-12-31')
              .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']);

// Function to calculate wind speed and direction
var addWindSpeedDirection = function(image) {
  var u = image.select('u_component_of_wind_10m');
  var v = image.select('v_component_of_wind_10m');
  var windSpeed = u.pow(2).add(v.pow(2)).sqrt().rename('wind_speed');
  var windDirection = u.atan2(v).rename('wind_direction');
  return image.addBands([windSpeed, windDirection]);
};

// Apply the wind speed and direction calculation to each image in the collection
var windCollection = era5.map(addWindSpeedDirection);

// Define the maximum number of images to export
var maxExports = 1100;  // 1464 - 365 = 1100 images in total
var exportCount = 0;

// Retrieve collection size
var windCollectionSize = windCollection.size().getInfo();
print('Wind Collection Size:', windCollectionSize);

// Set the start index and end index for exporting images
var startIndex = 365;
var endIndex = 1464;

if (windCollectionSize > 0) {
  // Retrieve collection as a list to access each image individually in smaller batches
  var imageList = windCollection.toList(windCollectionSize);

  // Loop through each image in the list and export in small batches, stopping at maxExports
  var batchSize = 5;

  for (var start = startIndex; start < endIndex && exportCount < maxExports; start += batchSize) {
    for (var i = 0; i < batchSize && (start + i) < endIndex && exportCount < maxExports; i++) {
      var image = ee.Image(imageList.get(start + i));
      Export.image.toDrive({
        image: image.select(['wind_speed', 'wind_direction']),
        description: 'Exported_Wind_Image_' + (start + i),
        scale: 1000,
        region: geometry,
        maxPixels: 1e9
      });
      exportCount++;  // Increment the count of exported images
    }
  }
  print('Total wind images exported:', exportCount);
} else {
  print("No images found in the specified date range and parameters.");
}
